In [1]:
%cd ../

/Users/aandres/source/protarrowrs


/Users/aandres/source/protarrowrs/env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import random
import secrets

import protarrow

import ptars
from ptars_protos.benchmark_pb2 import BenchmarkMessage

/Users/aandres/source/protarrowrs/env/lib/python3.11/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.2 at ptars_protos/benchmark.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


In [3]:
pool = ptars.HandlerPool()
handler = pool.get_for_message(BenchmarkMessage.DESCRIPTOR)

In [4]:
MESSAGE_COUNT = 10_000
STRING_SIZE = 10
MIN_INT = -2147483648
MAX_INT = 2147483647

In [5]:
messages = [
    BenchmarkMessage(
        query=secrets.token_urlsafe(random.randint(0, STRING_SIZE)),
        page_number=random.randint(MIN_INT, MAX_INT),
        result_per_page=random.randint(MIN_INT, MAX_INT),
    )
    for _ in range(MESSAGE_COUNT)
]

payloads = [m.SerializeToString() for m in messages]

In [6]:
%%timeit
protarrow.messages_to_record_batch(
    [BenchmarkMessage.FromString(p) for p in payloads],
    BenchmarkMessage,
)

14.2 ms ± 395 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
handler.list_to_record_batch(payloads)

86.6 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
record_batch = handler.list_to_record_batch(payloads)

In [9]:
%%timeit
handler.record_batch_to_array(record_batch)

68.4 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
[
    m.SerializeToString()
    for m in protarrow.record_batch_to_messages(record_batch, BenchmarkMessage)
]

38.6 ms ± 558 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
